# Human Activity Recognition using Inertial sensors and Neural Networks

**Elia Bonetto, Filippo Rigotto.**

Department of Information Engineering, University of Padova, Italy.

Human Data Analytics, a.y. 2018/2019

## Part 2 - DL models

TEST TO DO:
- augmented
- 70/30
- not-normalized

TO DO:
- bound on plot's y
- test various loss functions/lr/decay
- lr finder [link](https://medium.com/octavian-ai/how-to-use-the-learning-rate-finder-in-tensorflow-126210de9489)

NETWORKS:

DONE:
- FFNN (2,3 layers, with/without L2, with/without dropout)
- CNN (2,3 layers, with/without L2, with/without dropout)
- LSTM (2 layers)
- GRU (1 layer)

DOING:
- more CNNs
- LSTM
- RNN

TO DO:
- CNN + LSTM
- RF???
- AE
- Well known networks


In [0]:
!nvidia-smi

In [0]:
from IPython.display import Image, clear_output
import os
from google.colab import drive
drive.mount('/content/drive/')
clear_output()
os.chdir("/content/drive/My Drive/hda-project")
#!ls

In [0]:
!pip install telepot
clear_output()
from pprint import pprint
import json
from datetime import datetime
import pytz

import math
import h5py
import numpy as np
import scipy as sp
import scipy.io

import pandas as pd
pd.set_option('display.precision',3)
pd.set_option('display.float_format', '{:0.3f}'.format)

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical, plot_model
from tg_callback import TelegramCallback

#import logging
#logging.getLogger('tensorflow').disabled = True

from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')

## Data loading

Start from previously preprocessed data, altrady splitted in train and test parts.

In [0]:
map_decode = {
    0: 'running',
    1: 'walking',
    2: 'jumping',
    3: 'standing',
    4: 'sitting',
    5: 'lying',
    6: 'falling'
}
num_classes = len(map_decode)

In [0]:
with h5py.File('dataset/ARS-train-test-body-framed-aug-norm.h5','r') as h5f:
    X_train = h5f['X_train'][:] # IMU data w.r.t body frame
    X_test  = h5f['X_test'][:]  # activities (labels)
    Y_train = h5f['Y_train'][:]
    Y_test  = h5f['Y_test'][:]

num_data = len(X_train)
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("Y_test shape:  " + str(Y_test.shape))

# categorical structures are needed for the loss function to work properly
# original test classes are needed for prediction steps
Y_test_orig  = Y_test.copy()
Y_train = to_categorical(Y_train, num_classes=num_classes, dtype=np.uint8)
Y_test  = to_categorical(Y_test,  num_classes=num_classes, dtype=np.uint8)

## Training and evaluation

Precision, recall and F1 score are implemented in Tensorflow and are passed as metrics to track during training and evaluation of models.

The `run_model` function takes care of bootstrap, training and evaluation processes for a given Keras model and configuration.

In [0]:
def recall(y_true, y_pred):
    """Recall metric, batch-wise average."""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric, batch-wise average."""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    """F1 score, based on precision and recall metrics."""
    prc = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prc*rec)/(prc+rec+K.epsilon()))

def per_class_accuracy(y_true, y_preds, class_labels):
    # for reference. confusion matrix diag is used instead
    return [np.mean([
            (y_true[pred_idx] == np.round(y_pred)) 
                for pred_idx, y_pred in enumerate(y_preds) 
                    if y_true[pred_idx] == int(class_label)
        ]) for class_label in class_labels]

def halfLRafterEpoch(epoch):
    # for reference. lambda func is used instead
    initial_lrate = 0.1
    drop_rate = 0.5
    epochs_drop = 10.0
    return initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))


In [0]:
def run_model(model, config):
    """Generic method to build a model, train and evaluate performances."""

    out_folder = os.path.join('output', datetime.now(pytz.timezone('Europe/Rome')).strftime('%y%m%d-%H%M%S')+'_'+model.name)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    
    # print and save model summary
    print('Summary')
    model.summary()
    with open(os.path.join(out_folder, 'summary.txt'),'w') as sfile:
        model.summary(print_fn=lambda x: sfile.write(x+'\n'))
    plot_model(model, to_file=os.path.join(out_folder, 'model.png'), show_shapes=True)

    # save config
    with open(os.path.join(out_folder, 'config.json'),'w') as cfile:
        json.dump(config, cfile, indent=2)

    # compile model
    model.compile(optimizer=config['optimizer'],
                  loss=config['loss'],
                  metrics=['accuracy', precision, recall, f1])

    # add requested callbacks for model
    callbacks = []
    
    # ModelCheckpoint may go here
    
    if config['lr_step'] > 0:
        # halves lr every lr_step epochs (default lr = 0.1)
        callbacks.append(LearningRateScheduler(
            lambda epoch: 0.1 * math.pow(0.5, math.floor((1+epoch)/config['lr_step'])),
            verbose=1))
        
    if config['early_stop'] > 0:
        # stop if val_loss does has not diminished after num epochs
        callbacks.append(EarlyStopping(patience=config['early_stop']))
        
    if config['tg']:
        # telegram notification when training stops
        callbacks.append(TelegramCallback(name=model.name))
    
    # train model, save final state and history
    print('\nTraining')
    history = model.fit(x=X_train, y=Y_train,
                        shuffle=config['shuffle'],
                        epochs=config['epochs'],
                        batch_size=config['batch_size'],
                        callbacks=callbacks if len(callbacks) > 0 else None,
                        validation_data=(X_test,Y_test))
    
    model.save(os.path.join(out_folder, 'model.h5'))
    
    with open(os.path.join(out_folder, 'history.json'),'w') as hfile:
        hpd = pd.DataFrame(history.history)
        json.dump(json.loads(hpd.to_json()), hfile, indent=2)

        #json.dump(history.history, hfile, indent=2)
        # native json module can't handle float32 objects
        # pandas can and is used as a preprocessor to json module
    
    # evaluate model, save results
    print('\nEvaluation')
    metrics = model.evaluate(x=X_test, y=Y_test)
    metrics = dict(zip(model.metrics_names, metrics)) # build a dict adding names
    
    # get predictions
    preds = model.predict(x=X_test)
    Y_pred = np.argmax(preds, axis=1)
    
    classes_num = list(map(str,range(num_classes))) # classes list as str integers
    classes = list(map_decode.values())
    metrics['classes'] = classes
    
    # build per-class metrics and confusion matrix
    
    cr = classification_report(Y_test_orig, Y_pred, output_dict=True)
    
    cm = confusion_matrix(Y_test_orig, Y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normalization

    acc_class = [cm[i,i] for i in range(num_classes)]
    prc_class = [cr[cl]['precision'] for cl in cr if cl in classes_num] # exclude avgs
    rec_class = [cr[cl]['recall']    for cl in cr if cl in classes_num]
    f1_class  = [cr[cl]['f1-score']  for cl in cr if cl in classes_num]
    
    metrics['acc-class'] = acc_class
    metrics['precision-class'] = prc_class
    metrics['recall-class'] = rec_class
    metrics['f1-class'] = f1_class
    metrics['averages'] = cr['macro avg']
    metrics['weighted-averages'] = cr['weighted avg']
    del metrics['averages']['support']
    del metrics['weighted-averages']['support']
    print()
    pprint(metrics)

    # conversion to pure python float before saving to json
    for item in metrics:
        if type(metrics[item]) == np.float64 or type(metrics[item]) == np.float32:
            metrics[item] = float(metrics[item])
            
    with open(os.path.join(out_folder, 'evaluation.json'),'w') as efile:
        json.dump(metrics, efile, indent=2)
        
    np.save(os.path.join(out_folder, 'confusion.npy'), cm)

    # plot and save loss, accuracy and metrics (precision, recall, f1)
    print('\nLoss, accuracy, metrics and cm plots')
    plt.figure()
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-loss')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')

    plt.figure()
    plt.plot(history.history['acc'], label='Training')
    plt.plot(history.history['val_acc'], label='Validation')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-accuracy')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')
    
    plt.figure()
    plt.plot(history.history['precision'], label='Precision Tr')
    plt.plot(history.history['val_precision'], label='Precision Val')
    plt.plot(history.history['recall'], label='Recall Tr')
    plt.plot(history.history['val_recall'], label='Recall Val')
    plt.plot(history.history['f1'], label='F1 Tr')
    plt.plot(history.history['val_f1'], label='F1 Val')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Metrics')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-metrics')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')
    
    plt.figure()
    sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-confusion')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')

## Models

Here we layout the Keras models of the analyzed architectures.

### Feed-forward

In [0]:
def TwoDense_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'TwoDense'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name += '-do{}'.format(dropout)
    
    model = Sequential(name=name)
    model.add(Flatten(input_shape=input_shape))
    
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=l2(l2_reg) if l2_reg else None))
    
    if dropout_rate: model.add(Dropout(dropout_rate))

    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
def ThreeDense_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'ThreeDense'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name += '-do{}'.format(dropout)
    
    model = Sequential(name=name)
    model.add(Flatten(input_shape=input_shape))
    
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=l2(l2_reg) if l2_reg else None))
    
    if dropout_rate:
        model.add(Dropout(dropout_rate))
                  
    model.add(Dense(256, activation='relu',
                    kernel_regularizer=l2(l2_reg) if l2_reg else None))
    
    if dropout_rate:
        model.add(Dropout(dropout_rate))
                      
    model.add(Dense(num_classes, activation='softmax'))
    return model

### Convolutional

In [0]:
def Conv1D_1C1D_model(input_shape, num_classes, l2_reg=None):
    name = 'Conv1D-1C1D'
    if l2_reg: name += '-reg{}'.format(l2_reg)

    return Sequential([
        Conv1D(64, 5, input_shape=input_shape, 
               kernel_regularizer=l2(l2_reg) if l2_reg else None), # shape == (batch, steps, channels)
        BatchNormalization(axis=1),
        Activation('relu'),
        MaxPooling1D(2),
        
        Flatten(),
        Dense(num_classes, activation='softmax')
    ], name=name)

In [0]:
def Conv1D_1C2D_model(input_shape, num_classes, l2_reg=None):
    name = 'Conv1D-1C2D'
    if l2_reg: name += '-reg{}'.format(l2_reg)

    return Sequential([
        Conv1D(64, 5, input_shape=input_shape,
              kernel_regularizer=l2(l2_reg) if l2_reg else None),
        BatchNormalization(axis=1),
        Activation('relu'),
        MaxPooling1D(2),
        
        Flatten(),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_reg) if l2_reg else None),
        Dense(num_classes, activation='softmax')
    ], name=name)

In [0]:
def Conv1D_2C1D_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'Conv1D-2C1D'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name +='-do{}'.format(dropout_rate)
        
    model = Sequential(name=name)
    model.add(Conv1D(64, 5, input_shape=input_shape,
                     kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Conv1D(32, 5,
                     kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
def Conv1D_2C2D_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'Conv1D-2C2D'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name +='-do{}'.format(dropout_rate)
        
    model = Sequential(name=name)
    model.add(Conv1D(64, 5, input_shape=input_shape,
                     kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Conv1D(32, 5,
                     kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    return model

### Recurrent

In [0]:
def TwoLSTM_model(input_shape, num_classes):
    return Sequential([
        LSTM(32, return_sequences=True,  stateful=False, batch_input_shape=input_shape),
        LSTM(32, return_sequences=False, stateful=False),
        Dense(num_classes, activation='softmax')
    ], name='TwoLSTM')

In [0]:
def OneGRU_model(input_shape, num_classes):
    return Sequential([
        GRU(32, input_shape=input_shape),
        Dense(num_classes)
    ], name='OneGRU')

## Tests

Here models are trained according to selected configuration on the dataset split.

The configuration is a dictionary that allow to set the model's parameters and callbacks:

- `optimizer`: the selected optimizer for training
- `loss`: type of loss to minimize
- `epochs` and `batch_size`
- `shuffle`: whether to shuffle data in batches or keep the same order
- `lr_step`: epochs after which the learning rate is halved. Set to 0 to disable.
(UNIMPLEMENTED: set to 'auto' to reduce the lr when val loss does not decrease.)
- `early_stop`: number of epochs after which to stop training if validation loss does not decrease anymore. Set to 0 to disable.
- `tg`: whether to enable Telegram notification when training finishes

Models that contain Dense or convolutional layers may use L2 regularization with the optional parameter `l2_reg`.

Some models apply dropout if `dropout_rate` is set.

### Feed-forward

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 10,
    'batch_size': 32,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = TwoDense_model(input_shape, num_classes)
#model = TwoDense_model(input_shape, num_classes, l2_reg=0.01)
#model = TwoDense_model(input_shape, num_classes, dropout_rate=0.3)
#model = TwoDense_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.3)
run_model(model, config)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 25,
    'batch_size': 32,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = ThreeDense_model(input_shape, num_classes)
#model = ThreeDense_model(input_shape, num_classes, l2_reg=0.01)
#model = ThreeDense_model(input_shape, num_classes, dropout_rate=0.3)
#model = ThreeDense_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.3)
run_model(model, config)

### Convolutional

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 50,
    'batch_size': 32,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2]) # valid for 1D models only
model = Conv1D_1C1D_model(input_shape, num_classes)
#model = Conv1D_1C1D_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 50,
    'batch_size': 32,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2]) # valid for 1D models only
model = Conv1D_1C2D_model(input_shape, num_classes)
#model = Conv1D_1C2D_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 50,
    'batch_size': 32,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2]) # valid for 1D models only
model = Conv1D_2C1D_model(input_shape, num_classes)
#model = Conv1D_2C1D_model(input_shape, num_classes, l2_reg=0.01)
#model = Conv1D_2C1D_model(input_shape, num_classes, dropout_rate=0.3)
#model = Conv1D_2C1D_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.3)
run_model(model, config)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 50,
    'batch_size': 32,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2]) # valid for 1D models only
model = Conv1D_2C2D_model(input_shape, num_classes)
#model = Conv1D_2C2D_model(input_shape, num_classes, l2_reg=0.01)
#model = Conv1D_2C2D_model(input_shape, num_classes, dropout_rate=0.3)
#model = Conv1D_2C2D_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.3)
run_model(model, config)

### Recurrent

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 100,
    'batch_size': 300,
    'shuffle': False,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (None, X_train.shape[1], X_train.shape[2])
model = TwoLSTM_model(input_shape, num_classes)
run_model(model, config)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'mae',
    'epochs': 100,
    'batch_size': 200,
    'shuffle': False,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (None, X_train.shape[-1])
model = OneGRU_model(input_shape, num_classes)
run_model(model, config)

## Tests in pure Tensorflow

### LSTM

In [0]:
# model definition

features = 32 # number of hidden layer's features

#batch = 1500 # TODO unused vars
#n_iters = 300
#tot_iters = Y_train.shape[0] * n_iters
#disp_iter = 1000

w = {
    'h' : tf.Variable(tf.random_normal([X_train.shape[2], features])),
    'o' : tf.Variable(tf.random_normal([features, Y_train.shape[1]], mean=1.0))
}
b = {
    'h' : tf.Variable(tf.random_normal([features])),
    'o' : tf.Variable(tf.random_normal([Y_train.shape[1]]))
}

def LSTM(X, w, b):
    # input processing
    X = tf.transpose(X,[1,0,2])         # (batch_size, steps, input)
    X = tf.reshape(X, [-1, X.shape[2]]) # (steps*batch, n_initial_"features")

    X = tf.nn.relu(tf.matmul(X, w['h']) + b['h'])
    X = tf.split(X, X_train.shape[1])
    
    # model
    l_1 = tf.contrib.rnn.BasicLSTMCell(features, forget_bias=1.0, state_is_tuple=True)
    l_2 = tf.contrib.rnn.BasicLSTMCell(features, forget_bias=1.0, state_is_tuple=True)    
    lstm = tf.contrib.rnn.MultiRNNCell([l_1,l_2], state_is_tuple=True)    
    
    # output
    out, state = tf.contrib.rnn.static_rnn(lstm, X, dtype=tf.float32)
    
    return tf.matmul(out[-1], w['o']) + b['o']

In [0]:
# copy input
X_train_2 = X_train.astype(np.float32)
Y_train_2 = Y_train.astype(np.float32)
X_test_2 = X_test.astype(np.float32)
Y_test_2 = Y_test.astype(np.float32)

# define a dataset object on input
ds_obj = tf.data.Dataset.from_tensor_slices((X_train_2, Y_train_2)).repeat().batch(300)
iter = ds_obj.make_one_shot_iterator()
x, y = iter.get_next()

prediction = LSTM(x, w, b)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prediction,1), tf.argmax(y,1)),tf.float32))

# losses, optimizer
lr = 0.0025
lambda_l = 0.0015

l2_norm = lambda_l * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
softmax_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction)) + l2_norm
adam = tf.train.AdamOptimizer(learning_rate=lr).minimize(softmax_cost)

In [0]:
# run training
test_log  = {'loss':[], 'acc':[]}
train_log = {'loss':[], 'acc':[]}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000): #epochs
        _, l, a = sess.run([adam, softmax_cost, accuracy])
        train_log['loss'].append(l)
        train_log['acc'].append(a)
        
        l,a = sess.run([softmax_cost, accuracy])
        test_log['loss'].append(l)
        test_log['acc'].append(a)
        #print("PERFORMANCE ON TEST SET: " + \
        #      "Batch Loss = {}".format(l) + \
        #      ", Accuracy = {}".format(a))
print('Reached {}'.format(max(test_log['acc'])))

# save stuff and plots
out_folder = os.path.join('output', datetime.now(pytz.timezone('Europe/Rome')).strftime('%y%m%d-%H%M%S')+'_LSTM-TF')
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

with open(os.path.join(out_folder, 'history.json'),'w') as hfile:
    json.dump({'training':train_log, 'validation':test_log}, hfile, indent=2)
    
plt.figure()
plt.plot(train_log['loss'], label='Training')
plt.plot( test_log['loss'], label='Validation')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.tight_layout()
fname = os.path.join(out_folder, 'plot-loss')
plt.savefig(fname+'.png')
plt.savefig(fname+'.pdf', format='pdf')

plt.figure()
plt.plot(train_log['acc'], label='Training')
plt.plot( test_log['acc'], label='Validation')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.tight_layout()
fname = os.path.join(out_folder, 'plot-accuracy')
plt.savefig(fname+'.png')
plt.savefig(fname+'.pdf', format='pdf')